## Notebook For Eclipse Software Deployment ##
<i>Written by David Story </i>

___
### Description
This notebook was written to be an easy-to-use plateform for deploying and implementing software to AWS cloud computing services and creating datasets for the Eclipse Megamovie project, as well as related computationally heavy research projects.

This notebook will automate a number of tasks for preparing your AWS EC2 instances and deploying software to those instances. This notebook will:

<ul style=squares>
    <li>Do thing</li>
    <li>Do thing</li>
    <li>Do thing</li>
    <li>Do thing</li>
    <li>Do thing</li>
</ul>

___
### Information on Provided Files ###
This repository contains the following folders:
<ul style=squares>
    <li>examples</li>
    <li>keys</li>
    <li>documentation</li>
    <li>images</li>
    <li>movies</li>
    <li>logs</li>
    <li>figures</li>
    <li>results</li>
</ul>


___
#### examples: ####
This folder provides example files that will allow you to understand how files are being fed to this notebook, and how they are formated, to help you understand how to generate your files to allow this notebook to run smoothly.

#### keys: ####
This folder should hold your ssh or .pem keys that you generated for your EC2 instance

#### documentation: ####
This should should hold the .csv that describes your EC2 instance names and attributes.

#### images: ####
This should hold any images generated.

#### movies: ####
This folder should hold any movies generated.

#### servers: ####
This folder should hold any logs generated relating to server performance, states, and program performance and states.

#### figures: ####
This folder should hold any figures generated.

#### results: ####
This folder should hold any results generated.

___
### Dependencies
This notebook requires the following dependenices
<ul style=squares>
    <li>Do thing</li>
    <li>Do thing</li>
    <li>Do thing</li>
    <li>Do thing</li>
    <li>Do thing</li>
</ul>

___

### Acquiring your AWS EC2 Instance names ###
The program will first need to be fed a .CSV file that has the following information in order in rows. In the folder 'Examples', there will be an example of how the file should be formated.
<ul style=squares>
    <li>Name of EC2 Instance</li>
    <li>PEM key name</li>
    <li>Server Location (Ex. us-west-2)</li>
    <li>Server Type (Ubuntu, Amazon, etc)</li>
    <li>Number of cores (2 , 4, 8, 72, etc)</li>
    <li>amount of RAM (in gb) (8, 16, 32, 64, etc)</li>
</ul>